<table>
<tr height="150px">
<th>
<img height="80px" margin="20px" src='https://www.archimedesai.gr/images/logo_en.svg' />
</th>
<th>
<img height="150px" src='https://stergioc.github.io/assets/img/logos.png' />
</th>
</tr>
</table>

<h1>Introduction to Machine Learning (Hands-on Tutorial)</h1>
<h3>Stergios CHRISTODOULIDIS & Maria Vakalopoulou</h3>


[![ML-tutorial](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/stergioc/BioMed-AI-Summer-School/blob/master/ML/ml-tutorial.ipynb)